In [ ]:
import snapatac2 as snap
# import scanpy as sc
import anndata as sc


import numpy as np
import pandas as pd

# import matplotlib.pyplot as plt


In [ ]:
snap.__version__

In [ ]:
def get_index_from_array(a, b):
    """get the indices of b elements in a array
    """
    return pd.Series(a).reset_index().set_index(0).loc[b]['index'].values
    
    

# RNA 

In [ ]:
%%time
f = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/superdupermegaRNA_hasraw_P21.h5ad'
adata_rna = sc.read(f, backed='r')
adata_rna = adata_rna[adata_rna.obs['Study'] == '2023 Multiome']
adata_rna

In [ ]:
rna_cells = adata_rna.obs.index.values
print(rna_cells.shape)
rna_cells

# ATAC

In [ ]:
%%time
f_snaps      = [
    '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/atac_fragments/frag_snap/ATAC_P21a.h5ad',
    '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/atac_fragments/frag_snap/ATAC_P21b.h5ad',
]
samples = ['P21a', 'P21b']
f_snap_set  = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/atac_fragments/snap_P21ab.h5ad'
f_snap_bin  = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/atac_fragments/snap_P21ab_500kbbin.h5ad'
f_snap_bin2 = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/atac_fragments/snap_P21ab_500kbbin_org.h5ad'
f_snap_bin3 = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/atac_fragments/snap_P21ab_500kbbin_org_v2.h5ad'

adatas = []
for sample, f_snap in zip(samples, f_snaps):
    adata = snap.read(f_snap) # , backed='r')
    adatas.append((sample, adata))

adatas = snap.AnnDataSet(adatas, filename=f_snap_set)

In [ ]:
# %%time
# bindata = snap.pp.add_tile_matrix(adatas, bin_size=500, inplace=False,
#                                   file=f_snap_bin,
#                                   ) 

In [ ]:
# datashadow
## cellname = f"{barcode}-1-{sample}-2023 Multiome-{condition_code}-0"

atac_cells = adatas.obs_names
sample_labels = np.array(adatas.obs['sample']) ##.values
atac_cells = np.array([f'{c}-{s}-2023 Multiome-9-0' for c,s in zip(atac_cells, sample_labels)])

both_cells = np.intersect1d(rna_cells, atac_cells)
both_cells_idx = get_index_from_array(atac_cells, both_cells)
print(rna_cells.shape, 
      atac_cells.shape, 
      both_cells.shape, 
      len(both_cells)/len(rna_cells))

In [ ]:
# bindatasub = bindata.to_memory()[both_cells_idx]
# bindatasub.write(f_snap_bin2)

# read bindatasub

In [ ]:
bindatasub = snap.read(f_snap_bin2) # , backed='r')
bindatasub

In [ ]:
%%time
# snap.tl.spectral(bindatasub, features='selected')
snap.tl.spectral(bindatasub, features=None)

In [ ]:
%%time
snap.tl.umap(bindatasub)

In [ ]:
%%time
snap.pp.knn(bindatasub, use_rep="X_spectral")

In [ ]:
# bindatasub
bindatasub.write(f_snap_bin3)

# Leiden 

In [ ]:
%%time
snap.tl.leiden(bindatasub, resolution=2)

In [ ]:
umapXY = bindatasub.obsm['X_umap']
cell_labels = adata_rna.obs['Subclass'].loc[both_cells]
dfplot = pd.DataFrame(umapXY)
dfplot['label'] = cell_labels.values
dfplot['leiden'] = bindatasub.obs['leiden']#.values
dfplot['cell'] = both_cells
# sex_indicators = (np.ravel(adata[both_cells,'Xist'].X.todense())>0) #obs

In [ ]:
snap.pl.umap(bindatasub, color=cell_labels, interactive=False)

In [ ]:
snap.pl.umap(bindatasub, color="leiden", interactive=False)

In [ ]:
# confmat = dfplot.groupby(['leiden', 'label']).size().unstack()
# confmat.to_csv('/u/home/f/f7xiesnm/v1_multiome/rna_atac_confmat_p21_l23.csv')
# confmat_norm = confmat.divide(confmat.sum(axis=1), axis=0)
# confmat_norm

In [ ]:
# dfplot.to_csv('/u/home/f/f7xiesnm/v1_multiome/P21_atac_umap.csv')

In [ ]:
# snap.pl.umap(bindatasub, color=sex_indicators, interactive=False, height=500)